<a href="https://colab.research.google.com/github/smruthi-sreenivas/Fine_Tuning_OpenAIAPI/blob/main/Customer_Support.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#Import libraries
import os
import pandas as pd
from openai import OpenAI
import json

In [8]:
#set up API key

os.environ["OPENAI_API_KEY"] = 'sk-YA'

In [5]:
# prepare the dataset

df = pd.read_csv('/content/Customer-Support.csv')
df.head()

,query,response
0,My order hasn't arrived yet.,We apologize for the inconvenience. Can you pl...
1,I received a damaged product.,We apologize for the inconvenience. Can you pl...
2,I need to return an item.,Certainly. Please provide your order number an...
3,I want to change my shipping address.,No problem. Can you please provide your order ...
4,I have a question about my bill.,We'd be happy to help. Can you please provide ...


In [6]:
# convert to jsonl
with open('customer_support.jsonl','w') as f:
  for _,row in df.iterrows():
    json_line = json.dumps({"messages":[{"role": "system","content": "You are a helpful assistant for Customer SUpport",
                                        "role": "user","content": row["query"],
                                         "role": "assistant", "content": row["response"]}]
                            })
    f.write(json_line + '\n')

In [9]:
# upload file for finetuning
client = OpenAI()
file = client.files.create(
  file=open("customer_support.jsonl", "rb"),
  purpose='fine-tune'
)

In [10]:
file

FileObject(id='file-Rj4XJpT6CsMnHRGfarKMmd', bytes=13481, created_at=1744479367, filename='customer_support.jsonl', object='file', purpose='fine-tune', status='processed', expires_at=None, status_details=None)

In [11]:
#finetune the model
client.fine_tuning.jobs.create(
    training_file=file.id,
    model = 'gpt-4o-mini-2024-07-18',
    suffix = 'customer_support_bot'
)

FineTuningJob(id='ftjob-l2lJ0YMufhhh0n6MxSGyyJnv', created_at=1744479466, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-aKvDrUEOep2Iq75lPC0hrHSf', result_files=[], seed=831172503, status='validating_files', trained_tokens=None, training_file='file-Rj4XJpT6CsMnHRGfarKMmd', validation_file=None, estimated_finish=None, integrations=[], metadata=None, method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto')), type='supervised'), user_provided_suffix='customer_support_bot')

In [12]:
#monitor and use finetuned model
client.fine_tuning.jobs.list()

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-l2lJ0YMufhhh0n6MxSGyyJnv', created_at=1744479466, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size=1, learning_rate_multiplier=1.8, n_epochs=3), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-aKvDrUEOep2Iq75lPC0hrHSf', result_files=[], seed=831172503, status='validating_files', trained_tokens=None, training_file='file-Rj4XJpT6CsMnHRGfarKMmd', validation_file=None, estimated_finish=None, integrations=[], metadata=None, method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size=1, learning_rate_multiplier=1.8, n_epochs=3)), type='supervised'), user_provided_suffix='customer_support_bot'), FineTuningJob(id='ftjob-DjLQKcMobYwqTz9wx7WNnrVd', created_at=1744453257, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:smruthi:g

In [14]:
#use the finetuned model ft:gpt-4o-mini-2024-07-18:smruthi:faq-finetuning:BLCkGyuN
completion = client.chat.completions.create(
  model="ft:gpt-4o-mini-2024-07-18:smruthi:faq-finetuning:BLCkGyuN",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "how to return an item?"}
    ]
)
print("Fine-tuned model response:", completion.choices[0].message.content)

Fine-tuned model response: Returning an item typically involves a few steps, and the process may vary depending on the retailer. Here’s a general guideline on how to return an item:

1. **Check the Return Policy**: Before initiating a return, review the retailer’s return policy. This information can usually be found on their website or on your receipt.

2. **Contact Customer Service (if necessary)**: If you have questions about the return process, reach out to customer service for assistance. 

3. **Prepare the Item**: 
   - Ensure the item is in its original condition with all packaging and tags intact (if required).
   - Gather all receipts, invoices, or order confirmations.

4. **Fill Out Return Form**: Some retailers provide a return form that you need to fill out and include with the return package. 

5. **Repackage the Item**: Pack the item securely in its original packaging, if possible.

6. **Choose a Return Method**: Depending on the retailer, you may have different options fo

In [15]:
completion = client.chat.completions.create(
  model="ft:gpt-4o-mini-2024-07-18:smruthi:faq-finetuning:BLCkGyuN",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "can i cancel any order?"}
    ]
)
print("Fine-tuned model response:", completion.choices[0].message.content)

Fine-tuned model response: Whether you can cancel an order depends on the store or service’s policies, the status of your order, and the terms under which you made the purchase. Most online retailers allow order cancellations within a certain time frame. It’s best to check the specific cancellation policy on the retailer’s website or contact their customer service for assistance.


In [16]:
completion = client.chat.completions.create(
  model="ft:gpt-4o-mini-2024-07-18:smruthi:faq-finetuning:BLCkGyuN",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Can I place oredr from India?"}
    ]
)
print("Fine-tuned model response:", completion.choices[0].message.content)

Fine-tuned model response: Yes, you can place orders from India, depending on the specific context. If you are referring to purchasing products from international websites, many e-commerce platforms ship to India. However, it's important to check the shipping policies, import duties, and ensure that the seller delivers to your location in India. If you have a specific site or product in mind, I can provide more tailored assistance.
